# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [128]:
#ваш код 
import pandas as pd
data = pd.read_csv('data\Advertising.csv')

**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [129]:
#ваш код 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


In [130]:
#ваш код 
data.isna().sum()

Unnamed: 0    0
TV            0
radio         0
newspaper     0
sales         0
dtype: int64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [131]:
#ваш код 
import numpy as np
X = data[['TV','radio','newspaper']]
y = data['sales']
# преобразуем в массивы
X = np.array(X)
y = np.array(y)

## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [132]:
import numpy as np

X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [133]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм координатного спуска для случая нормализованных данных:

**Задано:**

* $X=(x_{ij})$ - матрица наблюдений, размерностью $dim(X)=(n, m)$
* $N=1000$ - количество итераций

**Примечание:** *1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.*

**Алгоритм (математическая запись):**
* Создать нулевой вектор параметров $w_0=(0, 0,..., 0)^T$
* Для всех $t=1, 2, ..., N$ итераций:
    * Для всех $k = 1, 2,..., m$:
        * Фиксируем значение всех признаков, кроме $k$-ого и вычисляем прогноз модели линейной регрессии.Для этого исключаем признак $k$-ый из данных и $w_j$ из параметров при построении прогноза.
        Математически это можно записать следующим образом:

        $$h_i = \sum_{j=1}^{k-1} x_{ij}w_{j} + \sum_{j=k+1}^{m} x_{ij}w_j $$

        **Примечание:**
        
        *Обратите, что в данной записи текущий признак под номером $k$ не участвует в сумме.Сравните эту запись с классической записью прогноза линейной регрессии в случае нормированных данных (когда участвуют все признаки):*

        $$h_i = \sum_{j=1}^{m} x_{ij}w_{j}$$ 
        
        * Вычисляем новое значение параметра $k$-ого коэффициента: 
        $$w_k = \sum_{i=1}^{n} x_{ik} (y_i - h_i) = x_k^T(y-h) $$

    * Вычисляем значение функции потерь и сохраняем в историю изменения функции потерь (В оценке функции потерь участвуют все признаки):
        $$\hat{y_i} = \sum_{j=1}^{m}x_{ij}$$
        $$Loss_t = \frac{1}{n} \sum_{i=1}^{n}(y_i-\hat{y_i})^2$$
        
        или в векторном виде:
        
        $$\hat{y} = Xw$$
        $$Loss_t = \frac{1}{n}(y-\hat{y})^T(y-\hat{y})$$
    



**Алгоритм (псевдокод):**
```python

num_iters = #количество итераций
m = # количество строк в матрице X
n = # количество столбцов в матрице X
w = #вектор размера nx1, состояющий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum((X @ w) - y) ** 2)/(len(y))

```

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [134]:
# ваш код 
num_iters = 100 #количество итераций
m = X.shape[0] # количество строк в матрице X
n = X.shape[1] # количество столбцов в матрице X
w = np.zeros((n,1)) #вектор размера nx1, состоящий из нулей

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum(((X @ w) - y) ** 2)/(len(y))
print(w)

[[ 41.56217205]
 [110.13144155]
 [ 73.52860638]
 [ -0.55006384]]


In [135]:
w = np.zeros((n,1))
w

array([[0.],
       [0.],
       [0.],
       [0.]])

In [136]:
w = np.zeros(n)
w

array([0., 0., 0., 0.])

Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [137]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [138]:
#ваш код
X = data[['TV','radio','newspaper']]
X = np.array(X) # преобразуем в массив
X = ( X - np.mean(X, axis=0) ) / np.std(X, axis=0) # масштабируем столбцы

**Добавим единичный столбец**

In [139]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [140]:
#ваш код
def mse_error(y, y_pred):
    """
    функция mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения y и предсказывающие y_pred, и возвращающую значение mse
    """
    error = (y_pred - y)**2
    mse = 1/(len(y))*np.sum(error)
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [141]:
#ваш код
y_pred = np.mean(y) # вычисляем среднее значение по столбцу sales
print(mse_error(y, y_pred)) # вычисляем ошибку

27.085743750000002


**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [142]:
#ваш код

def lin_pred(X, w):
    """_функция, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов_

    Args:
        X (_type_): _матрице предикторов_
        w (_type_): _вектору весов линейной модели_

    Returns:
        _type_: _description_
    """
    y_pred = np.dot(X, w)
    return y_pred

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [148]:
#ваш код
def stoch_grad_step(X, y, w, train_ind, eta):
    """    функция для реализации шага стохастического градиентного спуска

    Args:
        X (_type_): матрица наблюдений
        y (_type_): вектор целевой переменной
        w (_type_): вектор коэффициентов (весов)
        train_ind: индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
        eta (_type_): шаг градиентного спуска
    """
    y_pred = lin_pred(X[train_ind], w)
    n = len(X[train_ind])
    step = (2 * X[train_ind] * (y_pred - y[train_ind])) / n
    w_new = w - eta*step
    return w_new

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте оновной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [152]:
# ваш код
def stochastic_gradient_descent(X, y, par0, eta, iterations, precision):
    """    функция реализации стохастического градиентного спуска_

    Args:
        X (_type_): _Матрицу признаков X
        y (_type_): _Целевую переменнную
        par (_type_): _Изначальную точку (веса модели)
        alpha (_type_): _Параметр, определяющий темп обучения
        iterations (_type_): _Максимальное число итераций
        eta (_type_): _Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу
    """
    distance = float('inf') # Инициализируем расстояние между векторами весов на соседних итерациях бесконечностью
    mse = [] # пустой список для фиксации ошибок
    i = 0 # счетчик итераций
    par = par0
    # цикл обучения
    while True:
        train_ind = np.random.randint(X.shape[0])
        mse.append(mse_error(y, lin_pred(X[train_ind], par)))
        par_new = stoch_grad_step(X, y, par, train_ind, eta)
        distance = np.linalg.norm( par - par_new )
        if i > iterations or distance < precision:
            break
        par = par_new
        i += 1
        np.dot
    return w, mse    

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [153]:
# ваш код
par = np.zeros((4,1))
alpha = 0.01
iter = 100000
eta = 0.001
w, mse = stochastic_gradient_descent(X, y, par, alpha, iter, eta)
print(w)
print(mse)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

**Выведите вектор весов, к которому сошелся метод.**

In [151]:
# ваш код
print(w)

[0. 0. 0. 0.]


**Выведите среднеквадратичную ошибку на последней итерации.**

In [147]:
# ваш код